# Download country lyrics

In [2]:
# from urllib.request import Request, urlopen, quote # Python 3
from urllib2 import Request, urlopen, quote # Python 2    
from bs4 import BeautifulSoup
from string import punctuation
from difflib import SequenceMatcher as sm # For comparing similarity of lyrics
import requests
import json
import pandas as pd
import genius as genius_api
genius = genius_api.Genius()

### Get a list of each year's top 50 country artists from Billboard.com

In [3]:
def getArtistsFromList(URL):    
    # URL is a Billboard top list,
    # e.g. https://www.billboard.com/charts/year-end/2006/top-country-artists
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser")    
    chart_items = html.find_all("div", class_="ye-chart-item__title")
    return [item.get_text().strip() for item in chart_items]            

In [4]:
# Get the top 50 country artists each year since 2006
top_artists_by_year = {}
for year in range(2006,2018):    
    url = "https://www.billboard.com/charts/year-end/{}/top-country-artists".format(year)
    top_artists_by_year[year] = getArtistsFromList(url)

### Use the Genius API to get each artist's lyrics
https://github.com/johnwmillr/GeniusLyrics

In [4]:
# We want to reject songs that have already been added to artist collection
def songsAreSame(s1, s2):    
    # Idea credit: https://bigishdata.com/2016/10/25/talkin-bout-trucks-beer-and-love-in-country-songs-analyzing-genius-lyrics/
    # Compare lyric content using SequenceMatcher
    seqA = sm(None, s1.lyrics, s2['lyrics'])
    seqB = sm(None, s2['lyrics'], s1.lyrics)
    return seqA.ratio() > 0.5 or seqB.ratio() > 0.5

def songInArtist(new_song):    
    # artist_lyrics is global (works in Jupyter notebook)
    for song in artist_lyrics[year]['artists'][-1]['songs']:
        if songsAreSame(new_song, song):
            return True
    return False
    

In [ ]:
# Use the Genius API to get each artist's lyrics
found_artists = {}
artist_lyrics = {}
for year in reversed(list(top_artists_by_year.keys())):
    print("\n******* {} *******".format(year))    
    artist_lyrics[year] = {'artists': []}
    for rank, name in enumerate(top_artists_by_year[year]):
        print("\n--------")
        try:
            artist_lyrics[year]['artists'].append({})
            if name not in found_artists:
                artist = genius.search_artist(name)
                artist_lyrics[year]['artists'][-1]['artist'] = artist.name
                artist_lyrics[year]['artists'][-1]['rank'] = rank+1                
                artist_lyrics[year]['artists'][-1]['songs'] = []
                for song in artist.songs:                
                    if not songInArtist(song):                
                        artist_lyrics[year]['artists'][-1]['songs'].append({})
                        artist_lyrics[year]['artists'][-1]['songs'][-1]['title'] = song.title
                        artist_lyrics[year]['artists'][-1]['songs'][-1]['album'] = song.album
                        artist_lyrics[year]['artists'][-1]['songs'][-1]['year'] = song.year
                        artist_lyrics[year]['artists'][-1]['songs'][-1]['lyrics'] = song.lyrics                
                        artist_lyrics[year]['artists'][-1]['songs'][-1]['art'] = song.song_art_image_url
                    else:
                        print("SKIPPING \"{}\", already found in artist collection.".format(song.title))
                found_artists[name] = (name, year, len(artist_lyrics[year]['artists'])-1)
            else:
                # Store reference to artist location in dict, if artist previously found
                artist_lyrics[year]['artists'][-1] = found_artists[name]
        except:
            print('Skipping "{}" due to error.'.format(name))
            
    # Write the JSON object to disk as a backup before each new year
    with open('lyrics_{}.json'.format(year), 'w') as outfile:
        json.dump(artist_lyrics, outfile)
        
# Final write of the JSON object
with open('artist_lyrics.json', 'w') as outfile:
    json.dump(artist_lyrics, outfile)


******* 2017 *******

--------
Searching for Sam Hunt...

Can't add song by Kirk Knight & Nyck Caution, artist must be Sam Hunt.
Song 1: "Body Like a Back Road"
Song 2: "Body Like a Back Road (15 in a 30 Tour Live)"
Song 3: "Bottle It Up"
Song 4: "Break Up in a Small Town"
Song 5: "Come Over"
Come Over already in Sam Hunt, not adding song.
Song 6: "Come over - live from spotify nyc"
Song 7: "Cop Car"
Cop Car already in Sam Hunt, not adding song.
Song 8: "Drinkin' Too Much"
Song 9: "Drinkin' Too Much (8pm)"
Song 10: "Ex to See"
Song 11: "Ex to See (15 in a 30 Tour Live)"
Song 12: "Goodbye"
Can't add song by Carrie Underwood, artist must be Sam Hunt.
Song 13: "High School Heart"
Song 14: "House Party"
Song 15: "I'm Down"
Can't add song by William Michael Morgan, artist must be Sam Hunt.
Song 16: "I Met A Girl"
Song 17: "Leave The Night On"
Song 18: "Make You Miss Me"
Song 19: "Raised On It"
Song 20: "Saturday Night"
Song 21: "Single for the Summer"
Song 22: "Speakers"
Song 23: "Take You

Song 20: "Grave"
Can't add song by Danielle Bradbery, artist must be Thomas Rhett.
Can't add song by Jason Aldean, artist must be Thomas Rhett.
Song 21: "I Feel Good"
Song 22: "In a Minute"
Song 23: "It Goes Like This"
Song 24: "Kiss Me Like a Stranger"
Song 25: "Learned it From the Radio"
Song 26: "Leave Right Now"
Song 27: "Life Changes"
Song 28: "Like It's the Last Time"
Song 29: "Make Me Wanna"
Song 30: "Marry Me"
Song 31: "Mine*"
Song 32: "Playing With Fire"
Song 33: "Playing with Fire (New Version)"
Song 34: "Renegades"
Can't add song by LoCash, artist must be Thomas Rhett.
Can't add song by Florida Georgia Line, artist must be Thomas Rhett.
Can't add song by Thomas Rhett & Kelsea Ballerini, artist must be Thomas Rhett.
Song 35: "Single Girl"
Song 36: "Sixteen"
Can't add song by Brantley Gilbert, artist must be Thomas Rhett.
Song 37: "Smooth Like the Summer"
Song 38: "Something to Do with My Hands"
Song 39: "Sorry for Partyin'"
Song 40: "South Side"
Song 41: "Star of the Show"
Ca

Song 62: "Just Some Love"
Song 63: "Kiss A Girl"
Song 64: "Lil' digger"
Song 65: "Little Bit Of Everything"
Song 66: "Live To Love Another Day"
Song 67: "Long Hot Summer"
Song 68: "Love's Poster Child"
Song 69: "Love We Got Goin'"
Song 70: "Lovin' On The Side"
Song 71: "Lucky Charm"
Song 72: "Making Memories Of Us"
Song 73: "Man Of The House"
Can't add song by John Mayer, artist must be Keith Urban.
Song 74: "My Heart Is Open"
Song 75: "My Last Name"
Song 76: "Nobody Drinks Alone"
Song 77: "Once In A Lifetime"
Song 78: "Only You"
Song 79: "Only You Can Love Me This Way"
Song 80: "Out On My Own"
Song 81: "Parallel Line"
Song 82: "Put You In A Song"
Song 83: "Raining On Sunday"
Song 84: "Raise Em Up"
Song 85: "Raise The Barn"
Song 86: "Red Camaro"
Song 87: "Right On Back To You"
Song 88: "Rollercoaster"
Song 89: "Romeo's Tune"
Song 90: "Same Heart*"
Song 91: "Shame"
Song 92: "She's Gotta Be"
Can't add song by Colt Ford, artist must be Keith Urban.
Song 93: "She's My 11"
Song 94: "Shine"


Song 147: "Time For Me To Come Home"
Can't add song by Cassadee Pope, artist must be Blake Shelton.
Song 148: "Turnin' Me On"
Song 149: "Two Step 'Round the Christmas Tree"
Song 150: "Underneath The Same Moon"
Song 151: "What I Wouldn't Give"
Song 152: "What's On My Mind"
Song 153: "When Somebody Knows You That Well"
Song 154: "When the Wine Wears Off"
Song 155: "White Christmas"
Song 156: "Who Are You When I'm Not Looking"
Song 157: "Why Me"
Song 158: "Wildflower"
Song 159: "Winter Wonderland"
Can't add song by Chloe Kohanski, artist must be Blake Shelton.
Can't add song by Ashley Monroe, artist must be Blake Shelton.
Song 160: "You Can't Make This Up"
Song 161: "You'll Always Be Beautiful"
Can't add song by Gwen Stefani, artist must be Blake Shelton.
Found 161 songs.

Done.
SKIPPING "Boys 'Round Here (Celebrity Mix)", already found in artist collection.
SKIPPING "Still Got A Finger", already found in artist collection.
SKIPPING "This Can't Be Good", already found in artist collection

Song 19: "Pull It Off"
Song 20: "Rockstars"
Song 21: "Setting the Night On Fire"
Song 22: "There Goes My Everything"
Song 23: "Thunder in the Rain"
Song 24: "Used to Love You Sober"
Song 25: "What Ifs"
Song 26: "What's Mine Is Yours"
Song 27: "Wide Open"
Found 27 songs.

Done.

--------
Searching for Kenny Chesney...

Song 1: "A Chance"
Song 2: "Ain't Back Yet"
Song 3: "Ain't Back Yet (Previously Unreleased)"
Song 4: "Ain't Ever Going Back Again"
Song 5: "Ain't That Love"
Song 6: "All I Need To Know"
Song 7: "All I Want for Christmas Is a Real Good Tan"
Song 8: "All the Pretty Girls"
Song 9: "A Lot Of Things Different"
Song 10: "Always Gonna Be You"
Song 11: "American Kids"
Song 12: "Angel Loved The Devil"
Song 13: "Another Beer In Mexico"
Song 14: "Another Friday Night"
Song 15: "Anything But Mine"
Song 16: "Away in a Manger"
Song 17: "A Woman Knows"
Song 18: "Back In My Arms Again"
Song 19: "Back Where I Come From"
Song 20: "Back Where I Come From - New Recording with Kenny's Band"
S

...

SKIPPING "We're All Alone (Duet with Josey Y Durval)", already found in artist collection.
SKIPPING "Wish I Were Only Lonley", already found in artist collection.

******* 2016 *******

--------

--------

--------

--------

--------

--------

--------

--------

--------

--------

--------

--------

--------

--------
Searching for Joey + Rory...

Skipping "Joey + Rory" due to error.

--------

--------

--------

--------

--------

--------

--------

--------

--------
Searching for Dan + Shay...

Skipping "Dan + Shay" due to error.

--------

--------
Searching for Cam...

Song 1: "A Different Kind Of Normal"
Can't add song by Iggy Azalea, artist must be Cam.
Song 2: "Burning House"
Song 3: "Cold In California"
Song 4: "Country Ain't Never Been Pretty"
Song 5: "Diane"
Song 6: "Disciple Me"
Song 7: "Down This Road"
Song 8: "Do You Love Me"
Can't add song by Lecrae, artist must be Cam.
Song 9: "El Shaddai"
Song 10: "Fully Loaded"
Can't add song by Łuxury, artist must be Cam.
Son

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7d692b82c015>", line 35, in <module>
    json.dump(artist_lyrics, outfile)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/__init__.py", line 180, in dump
    fp.write(chunk)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1828, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/john/Projects/song-lyrics/lib/python3

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/Users/john/Projects/song-lyrics/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    s

### Load and view the lyrics JSON object

In [6]:
data = json.load(open('lyrics_2017.json'))
artists = data['2017']['artists']
artists.pop(36); # Must have been an error while downloading

### Count truck frequency

In [17]:
d = pd.DataFrame()

In [23]:
help(d)

Help on DataFrame in module pandas.core.frame object:

class DataFrame(pandas.core.generic.NDFrame)
 |  Two-dimensional size-mutable, potentially heterogeneous tabular data
 |  structure with labeled axes (rows and columns). Arithmetic operations
 |  align on both row and column labels. Can be thought of as a dict-like
 |  container for Series objects. The primary pandas data structure
 |  
 |  Parameters
 |  ----------
 |  data : numpy ndarray (structured or homogeneous), dict, or DataFrame
 |      Dict can contain Series, arrays, constants, or list-like objects
 |  index : Index or array-like
 |      Index to use for resulting frame. Will default to np.arange(n) if
 |      no indexing information part of input data and no index provided
 |  columns : Index or array-like
 |      Column labels to use for resulting frame. Will default to
 |      np.arange(n) if no column labels are provided
 |  dtype : dtype, default None
 |      Data type to force. Only a single dtype is allowed. If No

In [31]:
d.keys()

Index([u'artist', u'rank', u'songs'], dtype='object')

In [32]:
d

,artist,rank,songs
0,Sam Hunt,1.0,"[{u'album': u'NOW That's What I Call Music, Vo..."
1,Chris Stapleton,2.0,"[{u'album': u'From A Room: Volume 2', u'year':..."


In [99]:
pd.DataFrame(np.reshape(np.array([1,2,3]),(1,3)),columns=['Frequency','Count','Total'], index=[artist['artist']])

,Frequency,Count,Total
Reba McEntire,1,2,3


In [100]:
np.reshape([1,2,3],(3,1))

array([[1],
       [2],
       [3]])

In [108]:
def calcFreqOfTerm(artist, term="truck"):
    # Determine how many songs mention a given term
    songs = artist['songs']
    song_count, term_count = len(songs), 0
    for song in songs:
        if term.lower() in song['lyrics'].lower():
            term_count += 1        

    term_freq = round(term_count/float(song_count),5)
#     print('"{}" is mentioned in {:05.2f} percent ({:03}/{:03}) of songs by {}'.format(
#         term, 100*term_freq, term_count, song_count, artist['artist']))
    
    df = pd.DataFrame(np.reshape([term_freq, term_count, song_count],(1,3)),
                      columns=['Frequency','Count','Total'], index=[artist['artist']])
    df.name = term
    return df

f = calcFreqOfTerm(artists[0])
f

,Frequency,Count,Total
Sam Hunt,0.16667,4.0,24.0


### How often does each artist mention a given term?

In [117]:
truck = pd.concat([calcFreqOfTerm(artist, 'truck') for artist in artists],axis=0)
beer  = pd.concat([calcFreqOfTerm(artist, 'beer')  for artist in artists],axis=0)

In [125]:
truck

,Frequency,Count,Total
Sam Hunt,0.16667,4.0,24.0
Chris Stapleton,0.02778,1.0,36.0
Thomas Rhett,0.16000,8.0,50.0
Florida Georgia Line,0.13846,9.0,65.0
Luke Combs,0.16667,3.0,18.0
Brett Young,0.00000,0.0,17.0
Keith Urban,0.05634,8.0,142.0
Blake Shelton,0.11392,18.0,158.0
Jon Pardi,0.13793,4.0,29.0
Luke Bryan,0.22901,30.0,131.0


In [124]:
beer.head()

,Frequency,Count,Total
Sam Hunt,0.00000,0.0,24.0
Chris Stapleton,0.00000,0.0,36.0
Thomas Rhett,0.14000,7.0,50.0
Florida Georgia Line,0.10769,7.0,65.0
Luke Combs,0.27778,5.0,18.0
